<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

<p>In the first cell, we will import and install all the relvant libraries.</p>

In [2]:
import bs4
import pandas as pd # library for data analsysis
import lxml
import requests
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
# uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge geocoder --yes

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          82 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages a

<p>In the next cell, we will extract the table from the Wikipedia page and place it in a pandas dataframe.</p>

In [4]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

raw_random_wikipedia_page=requests.get(wikipedia_link)
open('page.html', 'wb').write(raw_random_wikipedia_page.content)

with open('page.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')
    
match = []
for td in soup.findAll('td'):
    matchs = td.text
    match.append(matchs)

# define the dataframe columns
column_names = ['Postcode', 'Borough', 'Neighbourhood']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

posts = match[::3]
posts = posts[:len(posts)-12]
bors = match[1::3]
bors = bors[:len(bors)-11]
neighs = match[2::3]
neighs = neighs[:len(neighs)-11]

cnt = len(posts)

ranger = list(range(0, cnt))


for data in ranger:
    neighborhoods = neighborhoods.append({'Postcode': posts[data],
                                          'Borough': bors[data],
                                          'Neighbourhood': ' '.join(neighs[data].split())}, ignore_index=True)

    
neighborhoods.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<p>The next cell has the script for viewing the shape of the dataframe. The shape can be seen below the cell</p>

In [109]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[1]
    )
)

The dataframe has 12 boroughs and 3 neighborhoods.


<p>Now, we will add the latitudes and longitudes to the data frame</p>

In [ ]:
import geocoder
# initialize your variable to None
lat_lng_coords = None
lat=list()
long=list()

# loop until you get the coordinates
for dt in ranger:
    while(lat_lng_coords is None):
        g = geocoder.google(posts[dt]+', Toronto, Ontario')
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    lat.append(latitude)
    long.append(longitude)

neighborhoods.columns = ['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

for data in ranger:
    neighborhoods = neighborhoods.append({'Latitude': latitude[data],
                                          'Longitude': longitude[data]}, ignore_index=True)
neighborhoods.head()

In [111]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [112]:
# create map of Toronto using latitude and longitude values
map_t = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_t)  
    
map_t

KeyError: 'Latitude'